In [2]:
cd /Users/nbeshouri/Documents/Projects/Metis/Project\ 4

/Users/nbeshouri/Documents/Projects/Metis/Project 4


In [3]:
from hwmf import utils, data
import re
import gensim
from gensim.models import doc2vec

In [96]:
name_to_bio_lemmas = utils.load_data('character_bios_lemmatized.pickle')
name_to_bio_tokens = utils.load_data('character_bios_tokenized.pickle')

In [142]:
name_to_article = data.get_character_articles()

In [ ]:
[(key, value) for key, value in name_to_bio_tokens.items() if key in name_to_article]

In [151]:
# This is a hacky think to filter out the stuff that should have been removed by now.
name_to_bio_lemmas = dict((key, value) for key, value in name_to_bio_tokens.items() if key in name_to_article) 

In [126]:
gensim.utils.simple_preprocess("that's how we do it!\nyeah man")

['that', 'how', 'we', 'do', 'it', 'yeah', 'man']

In [112]:
title_black_list = utils.load_values('character_article_title_black_list.csv')
word_black_list = utils.load_values('character_article_word_black_list.csv')

name_to_article = name_to_bio_lemmas

def merge_sents(sents):
    return ' '.join(sents)

def remove_black_listed(doc_string):
    patterns = [r'<.+?>']
#     patterns = []
    black_list = sorted(word_black_list, key=lambda x: -len(x))
    for pattern in word_black_list:
        patterns.append(r'\b' + pattern + '\b')
    patterns += word_black_list
    for pattern in patterns:
        doc_string = re.sub(pattern, '', doc_string)
    return doc_string

def to_word_list(sents):
    merged = merge_sents(sents)
    merged = remove_black_listed(merged)
    return gensim.utils.simple_preprocess(merged)

# def to_word_list(sents):
#     merged = merge_sents(sents)
#     merged = remove_black_listed(merged)
#     split = merged.split(' ')
#     return [word.strip() for word in split if word.strip()]

train_corpus = []
count = 0
for name, bio_sents in name_to_article.items():
    merged = to_word_list(bio_sents)
    doc = doc2vec.TaggedDocument(merged, [name])
    train_corpus.append(doc)
    count += 1
    if count % 1000 == 0:
        print(count)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000


In [113]:
to_word_list(name_to_article['Bender (Futurama)'])

['accord',
 'back',
 'build',
 'high',
 'tech',
 'industrial',
 'metalworking',
 'robot',
 'build',
 'manufacturing',
 'facility',
 'unlike',
 'robot',
 'mortal',
 'accord',
 'calculation',
 'year',
 'live',
 'after',
 'report',
 'defect',
 'manufacturer',
 'barely',
 'escape',
 'death',
 'guide',
 'missile',
 'robot',
 'death',
 'squad',
 'dispatch',
 'mom',
 'order',
 'eliminate',
 'effectively',
 'defective',
 'product',
 'market',
 'job',
 'metalworking',
 'fy',
 'bend',
 'steel',
 'girder',
 'construction',
 'suicide',
 'booth',
 'apartment',
 'although',
 'pair',
 'enjoy',
 'live',
 'portray',
 'manipulate',
 'guileless',
 'friend',
 'in',
 'early',
 'show',
 'prefer',
 'occupy',
 'small',
 'area',
 'apartment',
 'like',
 'closet',
 'refer',
 'cozy',
 'later',
 'show',
 'individual',
 'bedroom',
 'like',
 'hat',
 'magnet',
 'near',
 'pathological',
 'fear',
 'electric',
 'opener',
 'magnet',
 'because',
 'uncontrollably',
 'start',
 'sing',
 'folk',
 'song',
 'near',
 'head',
 'm

In [153]:
# model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=50)
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, window=10, epochs=50)
model.build_vocab(train_corpus)

In [154]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=5000)

CPU times: user 10h 5min 21s, sys: 29min 38s, total: 10h 35min
Wall time: 4h 35min 34s


In [159]:
gensim.models.doc2vec.FAST_VERSION

1

In [158]:
model.save('first_that_doesnt_suck.model')

In [4]:
model = utils.load_data('doc2vec_extracts.pickle')

In [5]:
# After training
model.docvecs.most_similar('Zuko')

[('Théoden', 0.6452826857566833),
 ('Lin Beifong', 0.5620329976081848),
 ('Taran (character)', 0.5428361296653748),
 ('Jacen Solo', 0.5262340903282166),
 ('Daughters of Finwë', 0.5107818245887756),
 ('Caliban', 0.5030316710472107),
 ('Ezio Auditore da Firenze', 0.49988019466400146),
 ('Rachel Summers', 0.4998725354671478),
 ('Keredic', 0.4998697340488434),
 ('Martine (character)', 0.4983367919921875)]

In [6]:
model.docvecs.most_similar('Yoda')

[('Luke Skywalker', 0.6885805130004883),
 ('Darth Bane', 0.6119592785835266),
 ('Obi-Wan Kenobi', 0.5899578332901001),
 ('Qui-Gon Jinn', 0.5864250659942627),
 ('Michael Williams (Henry V)', 0.5806146860122681),
 ('Mace Windu', 0.5779798030853271),
 ('Harry Potter (character)', 0.5710346698760986),
 ('Revan', 0.5558763742446899),
 ('Míster Danger', 0.546698272228241),
 ('Nightwolf', 0.5385563373565674)]

In [156]:
# After
model.docvecs.most_similar('Aang')

[('Bran Stark', 0.6570450067520142),
 ('Naruto Uzumaki', 0.6490170955657959),
 ('Aerith Gainsborough', 0.6444741487503052),
 ('Maximilian Jenius', 0.6083365082740784),
 ('Kakashi Hatake', 0.5698568820953369),
 ('Goku', 0.5684899091720581),
 ('Lelouch Lamperouge', 0.563614010810852),
 ('Paul Atreides', 0.5528197288513184),
 ('Boruto Uzumaki', 0.5497936606407166),
 ('Momiji (Ninja Gaiden)', 0.5424197316169739)]

In [137]:
model.docvecs.most_similar('Aang')

[('Aerith Gainsborough', 0.6787601709365845),
 ('Bran Stark', 0.5986438393592834),
 ('Goku', 0.5943489670753479),
 ('Gohan', 0.5703436136245728),
 ('Anna (Frozen)', 0.5645865201950073),
 ('Lelouch Lamperouge', 0.5546486377716064),
 ('Count Dooku', 0.5480912923812866),
 ('Paul Atreides', 0.545677900314331),
 ('Kyle Reese', 0.5450496673583984),
 ('Korra', 0.5447707176208496)]

In [139]:
model.docvecs.most_similar('Yoda')

[('Qui-Gon Jinn', 0.7101578712463379),
 ('Luke Skywalker', 0.6712669730186462),
 ('Optimus Prime', 0.6421325206756592),
 ('Vegeta', 0.6367337703704834),
 ('Hao Asakura', 0.6160750389099121),
 ('Obi-Wan Kenobi', 0.6145543456077576),
 ('Raoh', 0.5810256004333496),
 ('Darth Bane', 0.5740638971328735),
 ('Sephiroth (Final Fantasy)', 0.5709008574485779),
 ('Aqua (Kingdom Hearts)', 0.5666394829750061)]

In [157]:
# After
model.docvecs.most_similar('Yoda')

[('Qui-Gon Jinn', 0.6633567810058594),
 ('Luke Skywalker', 0.6167474985122681),
 ('Optimus Prime', 0.6107190847396851),
 ('Scourge (Transformers)', 0.5825558304786682),
 ('Kanan Jarrus', 0.5685822367668152),
 ('Zuko', 0.5668113231658936),
 ('Gimli (Middle-earth)', 0.565727710723877),
 ('Mokujin', 0.5602508783340454),
 ('Albus Dumbledore', 0.5537698268890381),
 ('Beorn', 0.5510784387588501)]

In [68]:
def find_most_similar(word_list):
    inferred_vector = model.infer_vector(word_list)
#     print(len(inferred_vector))
    sims = model.docvecs.most_similar([inferred_vector], topn=10)
    return sims

In [69]:
word_list = to_word_list(name_to_article['Bender (Futurama)'])
find_most_similar(word_list)

[('Bender (Futurama)', 0.9172766208648682),
 ('Burt Gummer', 0.6373168230056763),
 ('Sammy the Shunter', 0.6062910556793213),
 ('The Smurfs (comics)', 0.5927327275276184),
 ('Karla (character)', 0.5844019055366516),
 ('Nick Carter-Killmaster', 0.5783843398094177),
 ('Mega Mindy', 0.5779253840446472),
 ('Yig', 0.5778729915618896),
 ('Jerom', 0.5753814578056335),
 ('Pluk van de Petteflet', 0.5753750801086426)]

In [18]:
for doc in train_corpus_docs:
    page_name = doc.tags[0]
    inferred_vector = model.infer_vector(train_corpus[page_name].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=5
    print('Character:', page_name, 'Most Similar:'
    rank = [docid for docid, sim in sims].index(page_name)
    print(rank)
    break

0


In [165]:
from math import sin, cos, radians

def bench():
    product = 1.0
    for counter in range(1, 1000, 1):
        for dex in list(range(1, 360, 1)):
            angle = radians(dex)
            product *= sin(angle)**2 + cos(angle)**2
    return product
%timeit bench()

238 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
